In [ ]:
manager.trigger_nas_processing()

In [ ]:
# duck db
import duckdb

In [ ]:
conn = duckdb.connect(database="/mnt/AI_NAS/datalake/catalog/**/*.parquet", read_only=True)

In [ ]:
result = conn.execute("""
    SELECT * FROM read_parquet('/mnt/AI_NAS/datalake/catalog/provider=*/dataset=*/task=*/variant=*/**/data.parquet', union_by_name=True)
""").fetchdf()

In [ ]:
from openai import OpenAI
import time

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:9064/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [ ]:
# print model

models = client.models.list()
for model in models.data:
    print(model.id)

In [ ]:
import base64

def base64_encode(
    image_path,
):
    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()
    return base64.b64encode(image_bytes).decode("utf-8")
image_path = "/home/kai/workspace/DeepDocs_Project/datalake/managers/test4.jpg"
base64_image = base64_encode(
    image_path,
)
    
messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Extract all layout elements. Reading order does not matter.",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        },
    ]

In [ ]:


start_time = time.time()
completion = client.chat.completions.create(
    model="adapter",
    messages=messages,
    temperature=0.0,
)
end_time = time.time()
elapsed_time = end_time - start_time
tokens = completion.usage.total_tokens
tokens_per_second = tokens / elapsed_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Tokens: {tokens}")
print(f"Tokens per second: {tokens_per_second:.2f}")

In [ ]:
result = completion.choices[0].message.content

In [ ]:
target

In [ ]:
from transformers import Qwen2VLImageProcessor

In [ ]:
processor = Qwen2VLImageProcessor.from_pretrained("/mnt/AI_NAS/AI_MODEL/checkpoints/Qwen/Qwen2.5-VL-3B-Instruct")

In [ ]:
import numpy as np
image = Image.open(image_path)
inputs = processor(
    images=[image],
    return_tensors="np",
)

In [ ]:
inputs['pixel_values']

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import json
image = Image.open(image_path)
draw = ImageDraw.Draw(image)
target = json.loads(result)

for item in target:
    class_ = item.get("type", "unknown")
    bbox = item.get("bbox", [0, 0, 0, 0])
    x0, y0, x1, y1 = bbox
    x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)
    # 
    x0 -= 14
    x1 += 14
    y0 -= 14
    y1 += 14
    draw.rectangle([x0, y0, x1, y1], outline="red", width=2)
    draw.text((x0, y0), class_, fill="red")    

In [3]:
from datalake_client import DatalakeClient
from datasets import Dataset, load_from_disk
import os
from datetime import datetime
from glob import glob
import pandas as pd
# datalake_client.py
manager = DatalakeClient(
    nas_api_url="http://192.168.20.62:8000")

2025-06-16 11:34:19,783 - datalake_client - INFO - ✅ NAS API 서버 연결 확인: http://192.168.20.62:8000


In [6]:
manager.show_nas_dashboard()


📊 NAS Data Processing Dashboard
📦 Pending: 1개
🔄 Processing: 0개
❌ Failed: 0개
🖥️ Server Status: running
⏰ Last Updated: 2025-06-16T11:34:34.857913



In [5]:
parquets = glob("/mnt/AI_NAS/datalake/catalog/provider=*/dataset=funsd_plus/task=*/**/*.parquet", recursive=True)

In [7]:
parquets

['/mnt/AI_NAS/datalake/catalog/provider=huggingface/dataset=funsd_plus/task=vqa/variant=base_vqa/lang=en/src=real/data.parquet',
 '/mnt/AI_NAS/datalake/catalog/provider=huggingface/dataset=funsd_plus/task=kie/variant=base_kie/lang=en/src=real/data.parquet']

In [8]:

def apply_image_path(row):
    if row['image_path']:
        path = f"/mnt/AI_NAS/datalake/{row['image_path']}"
        # check path exists
        if os.path.exists(path):
            return path
        raise FileNotFoundError(f"Image path does not exist: {path}")
    return None
error_list = []
for parquet_path in parquets:
    try:
        db = Dataset.from_parquet(parquet_path)
        db = db.map(lambda x: {"image_path": apply_image_path(x)})
        # if db에 date가 없을경우
        if "date" not in db.column_names:
            db = db.map(lambda x: {"date": datetime.now().strftime("%Y-%m-%d")})
            
        sample_image_path = db[0]['image_path']
        
        provider, dataset, task, variant, *etc = parquet_path.replace("/mnt/AI_NAS/datalake/catalog/","").split("/")[:-1]
        provider = provider.replace("provider=", "")
        dataset = dataset.replace("dataset=", "")
        task = task.replace("task=", "")
        variant = variant.replace("variant=", "")
        meta = {}
        for et in etc:
            key, value = et.split("=")
            meta[key] = value
        if "table" in variant:
            meta["mod"] = "table"        

        print(f"provider: {provider}, dataset: {dataset}, task: {task}, variant: {variant}, meta: {meta}")
        temp_path = f"./temp/{provider}/{dataset}/{task}/{variant}"
        # db 에서 10개만 
        db = db.select(range(10))
        db.save_to_disk(temp_path)
        staging_dir, job_id = manager.upload_raw_data(
            data_file=temp_path,
            provider=provider,
            dataset=dataset,
            dataset_description="설명을 변경해주세요.",
            original_source="링크를 입력해주세요.",
        )
        # manager.trigger_nas_processing()
        # manager.wait_for_job_completion(job_id, timeout=1280000)
        # uploaded_db = Dataset.from_parquet(f"/mnt/AI_NAS/datalake/migrate_test/catalog/provider={provider}/dataset={dataset}/task=raw/variant=*/data.parquet")
        # uploaded_db.save_to_disk(temp_path)
        # manager.upload_task_data(
        #     data_file=temp_path,
        #     provider=provider,
        #     dataset=dataset,
        #     task=task,
        #     variant=variant,
        #     dataset_description="설명을 변경해주세요.",
        #     original_source="링크를 입력해주세요.",
        #     **meta,  # unpack meta dictionary
        # )
    except Exception as e:
        print(f"Error processing {parquet_path}: {e}")
        error_list.append({
            "parquet_path": parquet_path,
            "error": str(e)
        })

provider: huggingface, dataset: funsd_plus, task: vqa, variant: base_vqa, meta: {'lang': 'en', 'src': 'real'}


Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

2025-06-16 11:34:42,778 - datalake_client - INFO - 📥 Raw data 업로드 시작: huggingface/funsd_plus
2025-06-16 11:34:42,790 - datalake_client - INFO - 🗑️  기존 pending 데이터 정리: 1개 발견
2025-06-16 11:34:42,796 - datalake_client - INFO - 🗑️ 삭제 완료: funsd_plus_raw_image_8c0c9a07_20250616113208_kai
2025-06-16 11:34:42,797 - datalake_client - INFO - ✅ 기존 pending 데이터 정리 완료: 1개 삭제
2025-06-16 11:34:42,797 - datalake_client - INFO - 📂 데이터 파일 로드 중: /home/kai/workspace/DeepDocs_Project/datalake/managers/temp/huggingface/funsd_plus/vqa/base_vqa
2025-06-16 11:34:42,802 - datalake_client - INFO - ✅ datasets 폴더 로드 완료: 10 행
2025-06-16 11:34:42,802 - datalake_client - INFO - ✅ 데이터 파일 로드 완료: ./temp/huggingface/funsd_plus/vqa/base_vqa
2025-06-16 11:34:42,803 - datalake_client - INFO - 데이터셋 컬럼: ['image_path', 'query', 'label', 'width', 'height', 'date']
2025-06-16 11:34:42,803 - datalake_client - INFO - 🔍 JSON 변환 대상 컬럼 검사 시작
2025-06-16 11:34:42,805 - datalake_client - INFO - 📄 JSON 변환 대상 컬럼 없음
2025-06-16 11:34:42,807 

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

2025-06-16 11:34:42,907 - datalake_client - INFO - 📥 Raw data 업로드 시작: huggingface/funsd_plus
2025-06-16 11:34:42,914 - datalake_client - INFO - 🗑️  기존 pending 데이터 정리: 1개 발견
2025-06-16 11:34:42,920 - datalake_client - INFO - 🗑️ 삭제 완료: funsd_plus_raw_image_2479be4f_20250616113442_kai
2025-06-16 11:34:42,921 - datalake_client - INFO - ✅ 기존 pending 데이터 정리 완료: 1개 삭제
2025-06-16 11:34:42,922 - datalake_client - INFO - 📂 데이터 파일 로드 중: /home/kai/workspace/DeepDocs_Project/datalake/managers/temp/huggingface/funsd_plus/kie/base_kie
2025-06-16 11:34:42,927 - datalake_client - INFO - ✅ datasets 폴더 로드 완료: 10 행
2025-06-16 11:34:42,927 - datalake_client - INFO - ✅ 데이터 파일 로드 완료: ./temp/huggingface/funsd_plus/kie/base_kie
2025-06-16 11:34:42,928 - datalake_client - INFO - 데이터셋 컬럼: ['image_path', 'label', 'width', 'height', 'date']
2025-06-16 11:34:42,929 - datalake_client - INFO - 🔍 JSON 변환 대상 컬럼 검사 시작
2025-06-16 11:34:42,931 - datalake_client - INFO - 📄 JSON 변환 대상 컬럼 없음
2025-06-16 11:34:42,933 - datalak

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

2025-06-16 11:34:42,993 - datalake_client - INFO - 📦 datasets 저장 완료: /mnt/AI_NAS/datalake/migrate_test/staging/pending/funsd_plus_raw_image_9d3f0aec_20250616113442_kai
2025-06-16 11:34:42,994 - datalake_client - INFO - ✅ Task 데이터 업로드 완료: /mnt/AI_NAS/datalake/migrate_test/staging/pending/funsd_plus_raw_image_9d3f0aec_20250616113442_kai


In [16]:
staging_dir, job_id = manager.upload_raw_data(
    data_file=temp_path,
    provider=provider,
    dataset=dataset,
    dataset_description="설명을 변경해주세요.",
    original_source="링크를 입력해주세요.",
)

2025-06-16 11:24:09,255 - datalake_client - INFO - 📥 Raw data 업로드 시작: huggingface/funsd_plus
2025-06-16 11:24:09,277 - datalake_client - INFO - 📂 데이터 파일 로드 중: /home/kai/workspace/DeepDocs_Project/datalake/managers/temp/huggingface/funsd_plus/vqa/base_vqa
2025-06-16 11:24:09,286 - datalake_client - INFO - ✅ datasets 폴더 로드 완료: 10 행
2025-06-16 11:24:09,287 - datalake_client - INFO - ✅ 데이터 파일 로드 완료: ./temp/huggingface/funsd_plus/vqa/base_vqa
2025-06-16 11:24:09,289 - datalake_client - INFO - 데이터셋 컬럼: ['image_path', 'query', 'label', 'width', 'height', 'date']
2025-06-16 11:24:09,291 - datalake_client - INFO - 🔍 JSON 변환 대상 컬럼 검사 시작
2025-06-16 11:24:09,295 - datalake_client - INFO - 📄 JSON 변환 대상 컬럼 없음
2025-06-16 11:24:09,300 - datalake_client - INFO - 🔄 파일 컬럼 표준화: image_path → file_path
2025-06-16 11:24:09,304 - datalake_client - INFO - 📄 파일 분석 결과: variant=image, 이미지컬럼=[], 파일컬럼=['image_path'], 확장자={'.jpg'}


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

2025-06-16 11:24:09,405 - datalake_client - INFO - 📦 datasets 저장 완료: /mnt/AI_NAS/datalake/migrate_test/staging/pending/funsd_plus_raw_image_2b6841d0_20250616112409_kai
2025-06-16 11:24:09,407 - datalake_client - INFO - ✅ Task 데이터 업로드 완료: /mnt/AI_NAS/datalake/migrate_test/staging/pending/funsd_plus_raw_image_2b6841d0_20250616112409_kai


In [17]:
manager.trigger_nas_processing()

2025-06-16 11:24:10,311 - datalake_client - INFO - 🔄 NAS 처리 요청 중...
2025-06-16 11:24:10,321 - datalake_client - INFO - ✅ 처리 작업 시작됨: job_20250616_112410_319


'job_20250616_112410_319'

In [ ]:
manager.upload_raw_data(
    data_file="arrow경로",
    provider="provider_name",
    dataset="dataset_name",
    dataset_description="설명을 변경해주세요.",
    original_source="링크를 입력해주세요.",
)

In [ ]:
manager.upload_task_data(
    data_file=parquet_path,
    provider=provider,
    dataset=dataset,
    task=task,
    variant=variant,
    dataset_description="설명을 변경해주세요.",
    original_source="링크를 입력해주세요.",
    **meta,  # unpack meta dictionary
)

In [18]:
import duckdb

In [19]:
client = duckdb.connect()

In [60]:

result = client.execute("""
    SELECT * FROM read_parquet('/mnt/AI_NAS/datalake/migrate_test/catalog/provider=*/dataset=*/task=*/variant=*/data.parquet', union_by_name=True, filename=True, hive_partitioning=True)
""").fetchdf()

In [61]:
d = Dataset.from_pandas(result)

In [62]:
d[0]

{'query': 'This is a FAX COVER SHEET document. Please extract the following fields and return the result in the same JSON',
 'label': '{"FROM:": {"<|value|>": "Marco Giamberardino The Tobacco Institute 1875 I Street, N.W ., Suite 800 EPA Liga Washington, DC 20006 (202) 457-4800 (202) 457-9350 (FAX)", "<|bbox|>": [0.2248, 0.2745, 0.8297, 0.3908]}, "ID:": {"<|value|>": "2024579311", "<|bbox|>": [0.5748, 0.066, 0.6802, 0.0787]}, "PAGE": {"<|value|>": "1/4", "<|bbox|>": [0.8815, 0.0667, 0.9106, 0.0787]}, "TO:": {"<|value|>": "Mark Berlind", "<|bbox|>": [0.2267, 0.2156, 0.3547, 0.2326]}, "DATE:": {"<|value|>": "July 24, 1998", "<|bbox|>": [0.2248, 0.4085, 0.3471, 0.427]}, "Recipient\'s Fax Number:": {"<|value|>": "212-907-5805", "<|bbox|>": [0.3697, 0.7426, 0.4939, 0.7624]}}',
 'width': 1063,
 'height': 1410,
 'date': '2025-06-09',
 'path': 'provider=huggingface/dataset=funsd_plus/0c515a5ac5b055b3dfcd220bee34694fd1ac39c84d85cdc0cadf32aac07dd43e.jpg',
 'hash': '0c515a5ac5b055b3dfcd220bee3469

In [63]:
# path에 /mnt/AI_NASd/datalake/migrate_test/assets 추가
d = d.map(lambda x: {"path": f"/mnt/AI_NAS/datalake/migrate_test/assets/{x['path']}"})

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [64]:
# d의 filename이 exsts한지 확인
def check_image_exists(row):
    if row['path']:
        path = row['path']
        return os.path.exists(path)
    return False

In [65]:
d = d.map(lambda x: {"image_exists": check_image_exists(x)})

Map:   0%|          | 0/10 [00:00<?, ? examples/s]